**ALGORITHM 1: VL pretrained-M fine tuned with semi-disentangled outputs**

In [1]:
import requests
import os
import random
import json
import torch
import os
import kagglehub
import pickle
import math
import cv2

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

from PIL import Image
from collections import Counter
from sklearn.decomposition import PCA
from skimage.morphology import skeletonize
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel, AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split


from torchvision import transforms as A

In [15]:
# path = kagglehub.dataset_download("albertozorzetto/carla-densely-annotated-driving-dataset")
# print("Path to dataset files:", path)

Dataset Creation (IMAGE form to EMBEDDING form)

In [16]:
# # Download the CARLA dataset using the Kaggle API
# dataset_name = "albertozorzetto/carla-densely-annotated-driving-dataset"
# destination_path = "/content/datasets"

# import os
# # Install Kaggle API
# !pip install kaggle --upgrade
# # Upload kaggle.json for authentication
# from google.colab import files
# uploaded = files.upload()
# # Move kaggle.json to the proper directory
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# # Verify Kaggle is set up correctly
# !kaggle datasets list
# !kaggle datasets download -d {dataset_name} -p {destination_path} --unzip
# print("Path to dataset files:", destination_path)

# import tarfile
# images_path = os.path.join(destination_path, "images")
# labels_path = os.path.join(destination_path, "labels")
# # Create directories for extracted files
# os.makedirs(images_path, exist_ok=True)
# os.makedirs(labels_path, exist_ok=True)
# # Extract and manage .tar files
# tar_files = [f for f in os.listdir(destination_path) if f.endswith('.tar')]
# for tar_file in tar_files:
#     tar_path = os.path.join(destination_path, tar_file)
#     print(f"Extracting {tar_path}...")
#     with tarfile.open(tar_path) as tar:
#         # Determine the folder to extract based on the tar file name
#         if "images" in tar_file.lower():
#             tar.extractall(path=images_path)
#         elif "labels" in tar_file.lower():
#             tar.extractall(path=labels_path)
#         else:
#             print(f"Skipping unknown .tar file: {tar_file}")
#     os.remove(tar_path)  # Optional: Remove the .tar file after extraction
# print("All .tar files have been extracted.")
# print(f"Images folder: {images_path}")
# print(f"Labels folder: {labels_path}")

In [2]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
semantic_label_classes = ["Traffic Sign", "Building", "Fence", "Other", "Pedestrian", "Pole", "Road Line", "Road", "Sidewalk", "Vegetation", "Car", "Wall", "Unlabeled"]

root = "datasets"
images_dir = f"{root}/images"
labels_dir = f"{root}/labels"
waypoint_file = 'dataset_waypoints_try.json'
batch_size = 2

skip = 100

*Saving the {way points} for passing to the LLM*

In [55]:
def generate_waypoints(segmentation_map, num_waypoints=10):
    road_mask = (segmentation_map == 90).astype(np.uint8)
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return []
    largest_contour = max(contours, key=cv2.contourArea)
    binary_image = (road_mask > 0).astype(np.uint8)
    skeleton = skeletonize(binary_image).astype(np.uint8)
    y_coords, x_coords = np.where(skeleton > 0)
    path_points = np.column_stack((x_coords, y_coords))
    sampled_indices = np.linspace(0, len(path_points) - 1, num_waypoints, dtype=int)
    waypoints = path_points[sampled_indices]

    return waypoints.tolist()

In [62]:
def process_dataset(images_dir, output_dir, num_waypoints=10):
    os.makedirs(output_dir, exist_ok=True)
    dataset_waypoints = []

    for i in range(0, 28):
        k = "0" if i <= 9 else ""
        curr_folder_images = f"{images_dir}/Video_0{k}{i}"
        images_path = [file for file in os.listdir(curr_folder_images) if file.endswith(".png")]
        images_path.sort()
        # images_path = images_path[::skip]

        for image_file in images_path:
            image_path = os.path.join(curr_folder_images, image_file)
            image = Image.open(image_path)
            image = image.convert("L")
            image_np = np.array(image)
            waypoints = generate_waypoints(image_np, num_waypoints=num_waypoints)
            output_image_path = os.path.join(output_dir, image_file)
            dataset_waypoints.append({
                "image": output_image_path,
                "waypoints": waypoints
            })
            # break

        with open(os.path.join(output_dir, waypoint_file), 'a') as wf:
            json.dump(dataset_waypoints, wf)
            wf.write("\n")
            # break

        if(i<27):
            print(f"\rProgress: {((i + 1) / 0.27):.2f} %", end = "")
        # break

    print(f"\nDone")

In [ ]:
process_dataset(images_dir, root, num_waypoints=10)

*Fetching Data from images folder and dataset_waypoints.json*

In [3]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, labels_dir):
        self.data = []
        ei = []
        cw = []

        for i in range(0, 28):
            k = "0" if i <= 9 else ""
            curr_folder_images = f"{images_dir}/Video_0{k}{i}"
            images_path = [file for file in os.listdir(curr_folder_images) if file.endswith(".png")]
            images_path.sort()
            images_path = images_path[::skip]

            for image_file in images_path:
                image_path = os.path.join(curr_folder_images, image_file)
                image = Image.open(image_path)
                image_embeddings = processor(text=semantic_label_classes, images=image, return_tensors="pt", padding=True)
                ei.append(image_embeddings['pixel_values'])

            with open(os.path.join(root, waypoint_file), "r") as wf:
                content = wf.read()
                data_list = content.splitlines()
                for data in data_list:
                    data = json.loads(data)
                    for i in data:
                        corresponding_waypoints = i['waypoints']
                        cw.append(corresponding_waypoints)
                        # print(corresponding_waypoints)
                        # break
                    # break
            for e,c in zip(ei, cw):
                self.data.append((e,c))
                # break
            break
        # print(self.data[0][1])
        # print(self.data[0][0])
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        embedding_inputs, embedding_labels = self.data[idx]
        return embedding_inputs, embedding_labels

In [73]:
# Split indices for train and validation
dataset = SegmentationDataset(images_dir, labels_dir)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True, pin_memory=True)

train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

val_dataset = torch.utils.data.Subset(dataset, val_indices)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [8]:
def normalize(pixel_values):
    mini = pixel_values.min()
    maxi = pixel_values.max()
    pixel_values = (pixel_values - mini) / (maxi - mini)
    return pixel_values

OUR MODEL

In [70]:
class WaypointModel(nn.Module):
    def __init__(self):
        super(WaypointModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(512, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 64, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 40)

        self.dropout = nn.Dropout(0.25)

    def forward(self, pixel_values):
        x = torch.relu(self.conv1(pixel_values))
        x = self.bn1(x)
        x = torch.max_pool2d(x, 2)
        
        x = torch.relu(self.conv2(x))
        x = self.bn2(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv3(x))
        x = self.bn3(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv4(x))
        x = self.bn4(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv5(x))
        x = self.bn5(x)
        x = torch.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        
        output = self.fc2(x)
        return output.view(-1, 10, 2)

In [10]:
def to_float(val):
    return val.float()

In [11]:
def model_output_generation(pixel_values, model):
    pixel_values = pixel_values.squeeze(1)
    outputs = model(pixel_values)
    return outputs

*--training--*

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
# model = model.to(device)

In [71]:
model = WaypointModel()
criterion = nn.MSELoss()

In [15]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [16]:
def compute_accuracy(outputs, targets):
  threshold = targets.mean() * 0.01
  # print("outputs:", outputs)
  # print("targets:", targets)
  MSE = torch.sqrt(torch.sum((outputs - targets) ** 2, dim=-1))
  accurate_predictions = (MSE <= threshold).float()
  accuracy = accurate_predictions.mean().item()
  # print("MSE:", MSE)
  # print("acc preds:", accurate_predictions)
  # print("acc:", accuracy)
  return accuracy

In [67]:
def wayp_to_targets(way_p):
    # print(way_p)
    return to_float(torch.stack([torch.stack(pair, dim = 0) for pair in way_p]).permute(2,0,1))

In [ ]:
def test(model, dataloader, optimizer, criterion):
    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0

    with torch.no_grad():
        for i, (p_val, way_p) in enumerate(dataloader_1):
            target = wayp_to_targets(way_p)
            print(target)
            optimizer.zero_grad()

            outputs = model_output_generation(p_val[0], model)
            print(outputs)
            loss = criterion(outputs, target)
            running_loss += loss.item()

            accuracy = compute_accuracy(outputs, target)
            running_accuracy += accuracy
            
            # print(f"\r{running_loss:.1f}, {running_accuracy:.1f}")
            break

test(model, dataloader_1, optimizer, criterion)

In [73]:
def training(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0

    for i, (p_val, way_p) in enumerate(dataloader):
        target = wayp_to_targets(way_p)
        optimizer.zero_grad()

        outputs = model_output_generation(p_val[0], model)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        accuracy = compute_accuracy(outputs, target)
        running_accuracy += accuracy
        
        loss.backward()
        optimizer.step()
        # break

    return running_loss / len(dataloader), running_accuracy / len(dataloader)

In [74]:
def validation(model, dataloader, optimizer, criterion):
    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0

    with torch.no_grad():
        for i, (p_val, way_p) in enumerate(dataloader):
            target = wayp_to_targets(way_p)

            outputs = model_output_generation(p_val[0], model)
            loss = criterion(outputs, target)
            running_loss += loss.item()

            accuracy = compute_accuracy(outputs, target)
            running_accuracy += accuracy
            # break

    return running_loss / len(dataloader), running_accuracy / len(dataloader)

In [75]:
model = WaypointModel()
criterion = nn.MSELoss()
num_epochs = 50
lr = 1e-1

In [76]:
# Split indices for train and validation
dataset_1 = SegmentationDataset(images_dir, labels_dir)
dataloader_1 = DataLoader(dataset_1, batch_size = batch_size, shuffle = True, pin_memory=True)

train_indices, val_indices = train_test_split(range(len(dataset_1)), test_size=0.2, random_state=42)
train_dataset_1 = torch.utils.data.Subset(dataset_1, train_indices)
train_loader_1 = DataLoader(train_dataset_1, batch_size=batch_size, shuffle=True, pin_memory=True)

val_dataset_1 = torch.utils.data.Subset(dataset_1, val_indices)
val_loader_1 = DataLoader(val_dataset_1, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
best_model = None
for i in range(0,4):
  lr = lr / 10
  optimizer = optim.Adam(model.parameters(), lr=lr)
  prev_train_loss = float('inf')
  prev_val_loss = float('inf')
  # patience = 5
  print(f"Enhancing Learning rate at: {lr}")

  for epoch in range(num_epochs):
    best_model = model
    # if(patience == 0):
    #   break
    
    train_loss, train_accuracy = training(model, train_loader_1, optimizer, criterion)
    val_loss, val_accuracy = validation(model, val_loader_1, optimizer, criterion)
    
    if(train_loss > (prev_train_loss * 0.95) or (val_loss > (prev_val_loss * 0.95))):
      model = best_model
      # patience -= 1
    else:
      prev_train_loss = train_loss
      prev_val_loss = val_loss
      best_model = model
      # patience = 5
      # torch.save(model.state_dict(), '/content/drive/MyDrive/my_model.pth')
    print(f"\rEpoch {epoch + 1}/{num_epochs}, train-Loss: {train_loss:.1f}, val-Loss: {val_loss:.1f}     ", end = "")#, patience: {patience}", end = "")
    # break
  print("\n")
  # break

In [101]:
driver_model = model

In [81]:
torch.save(best_model.state_dict(), 'best_model.pth')

redundent code

In [1]:
import requests
import os
import random
import json
import torch
import os
import kagglehub
import pickle
import math
import cv2

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

from PIL import Image
from collections import Counter
from sklearn.decomposition import PCA
from skimage.morphology import skeletonize
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel, AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split


from torchvision import transforms as A

In [2]:
class WaypointModel(nn.Module):
    def __init__(self):
        super(WaypointModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(512, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 64, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 40)

        self.dropout = nn.Dropout(0.25)

    def forward(self, pixel_values):
        x = torch.relu(self.conv1(pixel_values))
        x = self.bn1(x)
        x = torch.max_pool2d(x, 2)
        
        x = torch.relu(self.conv2(x))
        x = self.bn2(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv3(x))
        x = self.bn3(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv4(x))
        x = self.bn4(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv5(x))
        x = self.bn5(x)
        x = torch.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        
        output = self.fc2(x)
        return output.view(-1, 10, 2)

In [3]:
def to_float(val):
    return val.float()

In [4]:
def model_output_generation(pixel_values, model):
    pixel_values = pixel_values.squeeze(1)
    outputs = model(pixel_values)
    return outputs

**CarLA simulation**

In [5]:
import carla
import random
import time
import kagglehub
import os
import torch
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import SegformerForSemanticSegmentation, SegformerConfig
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [6]:
driver_model = WaypointModel()
driver_model.load_state_dict(torch.load('best_model.pth', map_location=torch.device('cpu')))

C:\Users\devas\AppData\Local\Temp\ipykernel_18452\334242544.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  driver_model.load_state_dict(torch.load('best_model.pth', map

<All keys matched successfully>

In [7]:
def model_output_generation(pixel_values, model):
    pixel_values = pixel_values.squeeze(1)
    outputs = model(pixel_values)
    return outputs

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
semantic_label_classes = ["Traffic Sign", "Building", "Fence", "Other", "Pedestrian", "Pole", "Road Line", "Road", "Sidewalk", "Vegetation", "Car", "Wall", "Unlabeled"]

In [9]:
client = carla.Client('localhost', 2000)
world = client.get_world()
blueprint_library = world.get_blueprint_library()

netstat -an | findstr 2000
ping localhost

In [10]:
def remove_all_actors():
    actors = world.get_actors()
    for actor in actors:
        if 'vehicle' in actor.type_id:
            actor.destroy()
    world.tick()
    print("previous ones destroyed")

In [11]:
remove_all_actors()

previous ones destroyed


In [12]:
ego_blueprint = blueprint_library.filter('*model3')[0]
spawn_points = world.get_map().get_spawn_points()[0]
ego = world.spawn_actor(ego_blueprint, spawn_points)

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x = 1.5, z = 2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego)

In [13]:
def EGO_relative_view(EGO):
    view = EGO.get_transform().location + carla.Location(x=0, y=0, z=10)
    return view

In [14]:
def set_cam_to_EGO(EGO, angle = 55):
    spectator = world.get_spectator()
    vehicle_transform = EGO.get_transform()
    spectator.set_transform(
        carla.Transform(
            EGO_relative_view(EGO),
            carla.Rotation(pitch = -angle, yaw = vehicle_transform.rotation.yaw, roll = 0)
        )
    )

*functions to control*

In [15]:
def apply_action(vehicle, action):
    vehicle.apply_control(carla.VehicleControl(
        throttle=action['throttle'],
        steer=action['steer'],
        brake=action['brake']
    ))

In [68]:
def steering(x, intended_x):
    towards = intended_x - x
    steer = towards * 0.0004
    steer = max(-1, min(steer, 1))
    return steer

In [69]:
def TandB(y_mid, intended_y_mid, curr_y = 25):
    if(intended_y_mid <= curr_y):
        return 0, 1
    intended_y_mid = intended_y_mid if intended_y_mid < y_mid else y_mid
    return intended_y_mid / (1.75 * y_mid), 0

In [18]:
def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    return Image.fromarray(array[:, :, :3])

In [19]:
def image_callback(data):
    sensor_image = process_image(data)
sensor_image = None

In [20]:
def image_to_emb(image):
    global emb, image_1
    image_1 = image
    image = process_image(image)
    emb = processor(text=semantic_label_classes, images=image, return_tensors="pt", padding=True)

In [21]:
def normalize(way_p_1, top_w,top_h):
    way_p = way_p_1

    mini_f = float('inf')
    maxi_f = -float('inf')
    for i in way_p[0]:
        if(maxi_f < i[0]):
            maxi_f = i[0]
        if(mini_f > i[0]):
            mini_f = i[0]
        
    mini_s = float('inf')
    maxi_s = -float('inf')
    for i in way_p[1]:
        if(maxi_s < i[1]):
            maxi_s = i[1]
        if(mini_s > i[1]):
            mini_s = i[1]
        
    for i in way_p:
        i[0] = (i[0] - mini_f) / (maxi_f - mini_f)
        i[1] = (i[1] - mini_s) / (maxi_s - mini_s)
        i[0] *= top_h
        i[1] *= top_w

    return way_p

In [22]:
set_cam_to_EGO(ego)

*controlling based on model*

In [23]:
emb = None
prev_emb = None
image_1 = None

In [24]:
control = carla.VehicleControl()
control.steer = 0.0
control.throttle = 0.4
control.brake = 0.0
ego.apply_control(control)

In [25]:
camera.listen(image_to_emb)

while emb is None:
    world.tick()

In [26]:
camera.stop()

In [ ]:
prev_emb = emb.clone() if isinstance(emb, torch.Tensor) else emb
while True:
    ego.apply_control(control)
    if not torch.equal(prev_emb, emb):
        world.wait_for_tick()
        if prev_emb and not torch.equal(prev_emb, emb):
            i +=1
            print(f"\rchanged {i} times", end = "")
            prev_emb = emb

In [25]:
def print_controls(control):
    print(f"\r{control.steer}, {control.throttle}, {control.brake}", end = "")

In [70]:
# debug start

control = carla.VehicleControl()
control.steer = 0.0
control.throttle = 1
control.brake = 0.0
ego.apply_control(control)

start_time = time.time()
pe = {}
e = {}
idx = 0

set_cam_to_EGO(ego)
camera.listen(image_to_emb) #changes emb, image_1 globally

while emb is None:
    world.tick()

if prev_emb:
    for i in prev_emb.keys():
        pe[i] = prev_emb[i]
for i in emb.keys():
    e[i] = emb[i]

while True:
    world.wait_for_tick()
    e = emb['pixel_values']
    # print(f"unique emb['pixel_values]: {e}")
    if prev_emb:
        pe = prev_emb['pixel_values']
        # print(f"unique prev_emb['pixel_values]: {pe}")
        if not torch.equal(e,pe):

            outputs = model_output_generation(e, driver_model)
            waypoints = outputs.detach().numpy()

            scene_h, scene_w = process_image(image_1).size
            ego_x_mid = scene_h/2
            ego_y_mid = scene_w/2

            ego_x_position = ego_x_mid
            ego_y_position = 25

            new_wp = normalize(waypoints, scene_w, scene_h)
            intended_x_mid = np.mean([point[1] for point in waypoints])
            intended_y_mid = np.mean([point[0] for point in waypoints])

            steer = steering(ego_x_position, intended_x_mid)
            throttle, brake = TandB(ego_y_mid, intended_y_mid, ego_y_position)

            # print(outputs, waypoints, new_wp)
            print(f"\rrunning: at: ({ego_x_position:.1f}, {ego_y_position:.1f}), mid: ({ego_x_mid:.1f}, {ego_y_mid:.1f}), going towards: ({intended_x_mid:.1f}, {intended_y_mid:.1f}), with b:{brake}, t:{throttle:.1f}, s:{steer:.1f}", end = "")

            control = carla.VehicleControl()
            waypoints = outputs.detach().numpy()
            control.steer = steer
            control.throttle = throttle
            control.brake = brake
            ego.apply_control(control)
            set_cam_to_EGO(ego)
            
    if(time.time() >= start_time + 50):
        break
    prev_emb = emb

running: at: (960.0, 25.0), mid: (960.0, 540.0), going towards: (673.8, -27.8), with b:1, t:0.0, s:-0.11

In [71]:
# debug stop
print("stopped")

camera.stop()
remove_all_actors()

ego_blueprint = blueprint_library.filter('*model3')[0]
spawn_points = world.get_map().get_spawn_points()[0]
ego = world.spawn_actor(ego_blueprint, spawn_points)

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x = 1.5, z = 2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego)

set_cam_to_EGO(ego, 75)

stopped
previous ones destroyed
